In [47]:
import pandas as pd
import numpy as np
import json
import os
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from sklearn.metrics import confusion_matrix

from IPython.display import display
pd.set_option('display.max_rows',5)
pd.set_option('precision',3)

from IPython.core.display import HTML
HTML("<style>.container { width:100% !important;}</style>")

In [48]:
def json_to_df(file_name):
    #print(file_name)
    with open(file_name, encoding='utf-8', errors='ignore') as f:
        json_1 = json.load(f)
    dict_1 = dict(json_1)
    df_1 = pd.DataFrame.from_dict(dict_1)
    return df_1
def create_padded_docs(t,doc,max_length = 500):
    encoded_docs = t.texts_to_sequences(doc)
    #print(encoded_docs)
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    #print(padded_docs)
    return padded_docs

In [49]:
all_data_df['Id']=all_data_df.index
all_data_df

,index,text,sentiment,Id
0,1,Greetings and welcome to the Microsoft Fiscal ...,neutral,0
1,10,"With that as a backdrop, I want to highlight k...",positive,1
...,...,...,...,...
176,98,"Thanks, Mark. Operator, can we please move to ...",neutral,176
177,99,The next question is coming from the line of K...,neutral,177


In [50]:
import pickle
filename = 'all_group_data.pickle'
outfile = open(filename,'wb')
pickle.dump(all_data_df,outfile)
outfile.close()

In [191]:
all_data=[]
for file in os.listdir():
    
    #if file == "Team1_Google.json":
    #if file == "Team2_Amazon.json":
    #if file == "Team3_Facebook.json":
    #if file == "Team4_Netflix.json":    
    #if file == "Team5_Microsoft.json":
    #if file == "Team6_Tesla.json":
    #if file == "Team7_Walmart.json":
    #if file == "Team8_Kroger.json":
    #if file == "Team9_GoldmanSachs.json":
    #if file == "Team10_NVIDIA.json":
    #if file == "Team11_Boeing.json":
    #if file == "Team12_Chevron.json":

    if file.startswith("Team") and file.endswith(".json"):
        df=json_to_df(file)
        df.columns=['text','sentiment']
        df['sentiment']=df['sentiment'].apply(lambda x: x.lower())
        if sum(df.isnull().any(axis=1))>0:
            display(df[pd.isnull(df['Sentiment'])])
        else:
            all_data.append(df)
all_data_df=pd.concat(all_data)
all_data_df['sentiment']=all_data_df['sentiment'].apply(lambda x: 'neutral' if x in ['neutra;','negetive','neural'] else x)
all_data_df['sentiment']=all_data_df['sentiment'].apply(lambda x: 'positive' if x in ['postive','positivem'] else x)
print(all_data_df['sentiment'].unique())
display(all_data_df)
all_data_df_2type=all_data_df[all_data_df['sentiment'].isin(['positive','negative'])].copy()
all_data_df_2type=all_data_df.copy()

all_data_df=all_data_df.reset_index()
def encode(x):
    if x=='positive':
        return 2
    elif x=='neutral':
        return 1
    else:
        return 0
#all_data_df_2type_neu=all_data_df[~all_data_df['sentiment'].isin(['positive','negative'])].copy()
all_data_df_2type['label']=all_data_df_2type['sentiment'].apply(lambda x:encode(x))
all_data_df_2type['text_len']=all_data_df_2type['text'].apply(lambda x: len(x))
all_data_df_2type_pos=all_data_df_2type[all_data_df_2type['sentiment']=='positive'].copy()
all_data_df_2type_neg=all_data_df_2type[all_data_df_2type['sentiment']=='negative'].copy()
all_data_df_2type_neutral=all_data_df_2type[all_data_df_2type['sentiment']=='neutral'].copy()
print(all_data_df['sentiment'].unique())
all_data_df_2type


['neutral' 'negative' 'positive']


,text,sentiment
1,Could you catch us up on your latest thinking ...,neutral
10,We continue to ramp up the activities for the ...,neutral
...,...,...
98,And so it's -- that one is a little bit toughe...,positive
99,"Thanks, Simeon. Rob, next question please?",neutral


['neutral' 'negative' 'positive']


,text,sentiment,label,text_len
1,Could you catch us up on your latest thinking ...,neutral,1,305
10,We continue to ramp up the activities for the ...,neutral,1,560
...,...,...,...,...
98,And so it's -- that one is a little bit toughe...,positive,2,224
99,"Thanks, Simeon. Rob, next question please?",neutral,1,42


In [171]:
all_data_df_2type_neutral

,text,sentiment,label,text_len
1,Could you catch us up on your latest thinking ...,neutral,1,305
10,We continue to ramp up the activities for the ...,neutral,1,560
...,...,...,...,...
97,So as we step back and we looked at everything...,neutral,1,169
99,"Thanks, Simeon. Rob, next question please?",neutral,1,42


In [172]:
all_data_df_2type_neg

,text,sentiment,label,text_len
100,"Yes, it was sort of just how the conversation ...",negative,0,310
66,No. Thanks. We got it.,negative,0,22
...,...,...,...,...
89,"Thanks. Good morning, guys. And thanks for doi...",negative,0,311
91,The minority of the miss that was more operati...,negative,0,388


In [173]:
all_data_df_2type_neu

,index,text,sentiment
1,10,"During this call, we may discuss certain non-G...",neutral
2,11,Our guidance incorporates the order trends tha...,neutral
...,...,...,...
52,8,"Hello, and welcome to our Q4 2018 financial re...",neutral
53,9,Our comments and responses to your questions r...,neutral


In [174]:
y_train

,label
0,2
1,2
...,...
40,1
41,1


In [175]:
y_train_code

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [176]:
y_train['label'].unique()

array([2, 0, 1])

# 1. Problem 1

In [177]:
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(all_data_df_2type_pos[['text']], all_data_df_2type_pos[['label']], test_size=0.2, random_state=42)
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(all_data_df_2type_neg[['text']], all_data_df_2type_neg[['label']], test_size=0.2, random_state=42)
X_train_nu, X_test_nu, y_train_nu, y_test_nu = train_test_split(all_data_df_2type_neutral[['text']], all_data_df_2type_neutral[['label']], test_size=0.2, random_state=42)
#display(X_train_p, X_test_p, y_train_p, y_test_p)
#display(X_train_n, X_test_n, y_train_n, y_test_n)
X_train=X_train_p.append(X_train_n).append(X_train_nu).copy().reset_index(drop=True)
X_test=X_test_p.append(X_test_n).append(X_test_nu).copy().reset_index(drop=True)
y_train=y_train_p.append(y_train_n).append(y_train_nu).copy().reset_index(drop=True)
y_test=y_test_p.append(y_test_n).append(y_test_nu).copy().reset_index(drop=True)


from keras.utils import to_categorical
y_train_code=to_categorical(y_train['label'])
y_test_code=to_categorical(y_test['label']) 

docs = all_data_df['text'].tolist()
max_words = 10000 
t = Tokenizer(num_words=max_words)
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
for vector_len in [12]:
    for m_len in [100]:
        print("max length: "+str(m_len)+"   |   vector length: "+str(vector_len))
        x_train_doc=create_padded_docs(t,X_train['text'].tolist(),m_len)
        x_test_doc=create_padded_docs(t,X_test['text'].tolist(),m_len)
        y_train_label=y_train_code
        y_test_label=y_test_code

        model = Sequential()
        model.add(Embedding(vocab_size, vector_len, input_length=m_len))
        model.add(Flatten())
        model.add(Dense(3, activation='softmax'))
        # compile the model
        model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
        # summarize the model
        print(model.summary())
        # fit the model
        model.fit(x_train_doc, y_train_label, epochs=50)#,validation_split=0.1)
        # evaluate the model
        loss, accuracy = model.evaluate(x_train_doc, y_train_label, verbose=0)
        print('Training Accuracy: %f' % (accuracy*100))
        y_train_label_predicted=model.predict(x_train_doc)
        y_test_label_predicted=model.predict(x_test_doc)
        loss, accuracy = model.evaluate(x_test_doc, y_test_label, verbose=0)
        print('Testing Accuracy: %f' % (accuracy*100))
        print('***'*30)

max length: 100   |   vector length: 12
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 100, 12)           76524     
_________________________________________________________________
flatten_15 (Flatten)         (None, 1200)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 3603      
Total params: 80,127
Trainable params: 80,127
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
1315/1315 [==============================] - 0s 290us/step - loss: 0.9240 - acc: 0.5460
Epoch 2/50
1315/1315 [==============================] - 0s 42us/step - loss: 0.8076 - acc: 0.6654
Epoch 3/50
1315/1315 [==============================] - 0s 44us/step - loss: 0.7537 - acc: 0.7042
Epoch 4/50
1315/1315 [==============================] - 0s 40u

In [178]:
def find_label(x):
    if x['positive']>x['negative']:
        if x['positive']>=x['neutral']:
            return 2
        else:
            return 1
    else:
        if x['negative']>=x['neutral']:
            return 0
        else:
            return 1
y_train_label_predicted_df=pd.DataFrame(y_train_label_predicted)
y_train_label_predicted_df.columns=['negative','neutral','positive']        
y_train_label_predicted_df['predicted_label']=y_train_label_predicted_df.apply(lambda x: find_label(x),axis=1)
y_train_label_predicted_df[y_train_label_predicted_df['predicted_label']==0]

y_test_label_predicted_df=pd.DataFrame(y_test_label_predicted)
y_test_label_predicted_df.columns=['negative','neutral','positive']        
y_test_label_predicted_df['predicted_label']=y_test_label_predicted_df.apply(lambda x: find_label(x),axis=1)
y_test_label_predicted_df[y_test_label_predicted_df['predicted_label']==0]

,negative,neutral,positive,predicted_label
74,0.688,0.282,0.030,0
83,0.535,0.098,0.368,0
...,...,...,...,...
197,0.911,0.004,0.086,0
262,0.476,0.473,0.051,0


In [179]:
1644*0.2

328.8

In [180]:
confusion_columns=['predicted_neg','predicted_neutral','predicted_pos']
confusion_rows=['true_neg','true_neutral','true_pos']
train_confusion_matrix=pd.DataFrame(confusion_matrix(y_train['label'].tolist(), y_train_label_predicted_df['predicted_label']))
train_confusion_matrix.columns=confusion_columns
train_confusion_matrix['index']=confusion_rows
train_confusion_matrix=train_confusion_matrix.set_index('index')
train_confusion_matrix.index.name=''

test_confusion_matrix=pd.DataFrame(confusion_matrix(y_test['label'].tolist(), y_test_label_predicted_df['predicted_label']))
test_confusion_matrix.columns=confusion_columns
test_confusion_matrix['index']=confusion_rows
test_confusion_matrix=test_confusion_matrix.set_index('index')
test_confusion_matrix.index.name=''

display(train_confusion_matrix)
display(test_confusion_matrix)


,predicted_neg,predicted_neutral,predicted_pos
,,,
true_neg,123,0,1
true_neutral,1,665,2
true_pos,0,1,522


,predicted_neg,predicted_neutral,predicted_pos
,,,
true_neg,3,14,14
true_neutral,2,133,32
true_pos,4,42,85


# 2. Problem 2

In [30]:
import os

imdb_dir = 'aclImdb'
def load_imdb_dir(imdb_dir,sub_folder):
    train_dir = os.path.join(imdb_dir,sub_folder)
    labels = []
    texts = []
    for label_type in ['neg', 'pos']:
        dir_name = os.path.join(train_dir, label_type)
        for fname in os.listdir(dir_name):
            if fname[-4:] == '.txt':
                f = open(os.path.join(dir_name, fname))
                texts.append(f.read())
                f.close()
                if label_type == 'neg':
                    labels.append(0)
                else:
                    labels.append(1)
    return texts,labels
x_train_imdb_raw,y_train_imdb_raw=load_imdb_dir(imdb_dir,"train")
x_test_imdb_raw,y_test_imdb_raw=load_imdb_dir(imdb_dir,"test")


x_train_imdb_raw_tot=x_train_imdb_raw+x_test_imdb_raw
y_train_imdb_raw_tot=y_train_imdb_raw+y_test_imdb_raw

max_words = 10000 
docs = all_data_df_2type['text'].tolist()+x_train_imdb_raw_tot
t = Tokenizer(num_words=max_words)
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

In [31]:

for vector_len in [12]:
    for m_len in [100]:
        print("max length: "+str(m_len)+"   |   vector length: "+str(vector_len))
        x_train_doc=create_padded_docs(t,x_train_imdb_raw_tot,m_len)
        x_test_doc=create_padded_docs(t,all_data_df_2type['text'].tolist(),m_len)
        y_train_label=y_train_imdb_raw_tot
        y_test_label=all_data_df_2type['label'].tolist()

        model = Sequential()
        model.add(Embedding(vocab_size, vector_len, input_length=m_len))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        # compile the model
        model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
        # summarize the model
        print(model.summary())
        # fit the model
        model.fit(x_train_doc, y_train_label, epochs=5,validation_split=0.1)
        # evaluate the model
        loss, accuracy = model.evaluate(x_train_doc, y_train_label, verbose=0)
        print('Training Accuracy: %f' % (accuracy*100))
        y_train_label_predicted=model.predict(x_train_doc)
        y_test_label_predicted=model.predict(x_test_doc)
        y_train_label_predicted=[1 if y>=0.5 else 0 for y in y_train_label_predicted]
        y_test_label_predicted=[1 if y>=0.5 else 0 for y in y_test_label_predicted]
        loss, accuracy = model.evaluate(x_test_doc, y_test_label, verbose=0)
        print('Testing Accuracy: %f' % (accuracy*100))
        print('***'*30)

max length: 100   |   vector length: 12
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 12)           1491816   
_________________________________________________________________
flatten_4 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 1201      
Total params: 1,493,017
Trainable params: 1,493,017
Non-trainable params: 0
_________________________________________________________________
None
Train on 45000 samples, validate on 5000 samples
Epoch 1/5


KeyboardInterrupt: 

In [ ]:
all_data_df_2type['y_predicted']=y_test_label_predicted
all_data_df_2type['label_predicted']=all_data_df_2type['y_predicted'].apply(lambda x: 'positive' if x==1 else "negative")
all_data_df_2type_neu['label_predicted']='neutral'
all_data_df_2type=all_data_df_2type.drop('y_predicted',axis=1)
ge_df=all_data_df_2type.append(all_data_df_2type_neu)
ge_df.to_json('GE_predicted.json',orient='split')


In [ ]:
output='''{"text":{'''
senti='''},"sentiment":{'''
for index, row in ge_df.iterrows():
    output+='''"'''+str(index)+'''":"'''+row['text']+'''",'''
    senti+='''"'''+str(index)+'''":"'''+row['label_predicted']+'''",'''
output=output[:-1]
senti=senti[:-1]+'}}'
out_senti=output+senti
out_senti

In [ ]:
text_file = open("GE_predicted.json", "w")
text_file.write(out_senti)
text_file.close()

In [ ]:
confusion_columns=['predicted_neg','predicted_pos']
confusion_rows=['true_neg','true_pos']
train_confusion_matrix=pd.DataFrame(confusion_matrix(y_train_label, y_train_label_predicted))
train_confusion_matrix.columns=confusion_columns
train_confusion_matrix['index']=confusion_rows
train_confusion_matrix=train_confusion_matrix.set_index('index')
train_confusion_matrix.index.name=''

test_confusion_matrix=pd.DataFrame(confusion_matrix(y_test_label, y_test_label_predicted))
test_confusion_matrix.columns=confusion_columns
test_confusion_matrix['index']=confusion_rows
test_confusion_matrix=test_confusion_matrix.set_index('index')
test_confusion_matrix.index.name=''

display(train_confusion_matrix)
display(test_confusion_matrix)

# Problem 3

# IBM

In [97]:
!pip install --upgrade "watson-developer-cloud>=2.5.1"

X_raw = all_data_df['text'].tolist()
y_raw = all_data_df['sentiment'].tolist()
#y = keras.utils.to_categorical(y_raw, 3)
X = str(' '.join(X_raw))

print(len(X))
print(len(X_raw))

import json
import keras
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, SentimentOptions


natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-11-16',
    iam_apikey='JB9Bepy8qpZLVwVYSKobgClglbAcsHvkZRSzOHvm1CnM',
    url='https://gateway-wdc.watsonplatform.net/natural-language-understanding/api')
preds = []
count = 0
for x in X_raw:
    response = natural_language_understanding.analyze(
        text = str(x),
        features=Features(sentiment=SentimentOptions(document = True))).get_result()
    preds.append(list([count,list(response['sentiment'].values())]))
    
    count = count + 1;
    print(count)
    if count > 126:
        break

Requirement already up-to-date: watson-developer-cloud>=2.5.1 in /Users/lovelife/anaconda3/lib/python3.6/site-packages (2.8.1)


In [ ]:
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
    import Features, EntitiesOptions, KeywordsOptions

naturalLanguageUnderstanding = NaturalLanguageUnderstandingV1(
    version='2018-11-16',
    username='wenjun song',
    password='Ilove22*',
    #iam_apikey='OAbro0YFtCjxCW5Z8EiHjBcKGHtm2qpl30rvUw9yDZJ1',
    url='https://gateway-tok.watsonplatform.net/natural-language-understanding/api')

response = naturalLanguageUnderstanding.analyze(
    text=all_data_df_2type['text'].tolist()[0],
    features=Features(
        entities=EntitiesOptions(emotion=True, sentiment=True, limit=2),
        keywords=KeywordsOptions(emotion=True, sentiment=True,
                                 limit=2))).get_result()

print(json.dumps(response, indent=2))

# Google

In [18]:
!pip install --upgrade google-api-python-client
!easy_install --upgrade google-api-python-client
! pip install google-cloud-storage
! pip install google-cloud-language
! pip install --upgrade google-cloud-language
import argparse
import google.cloud
from google.cloud import language
from google.cloud import storage
from google.cloud.language import enums
from google.cloud.language import types
import os
from google.oauth2 import service_account
!pip install google-auth-oauthlib
from google_auth_oauthlib.flow import Flow
from google_auth_oauthlib.flow import InstalledAppFlow

Requirement already up-to-date: google-api-python-client in /Users/lovelife/anaconda3/lib/python3.6/site-packages (1.7.8)
Searching for google-api-python-client
Reading https://pypi.org/simple/google-api-python-client/
Best match: google-api-python-client 1.7.8
Processing google_api_python_client-1.7.8-py3-none-any.whl
removing '/Users/lovelife/anaconda3/lib/python3.6/site-packages/google_api_python_client-1.7.8-py3.6.egg' (and everything under it)
Installing google_api_python_client-1.7.8-py3-none-any.whl to /Users/lovelife/anaconda3/lib/python3.6/site-packages
writing requirements to /Users/lovelife/anaconda3/lib/python3.6/site-packages/google_api_python_client-1.7.8-py3.6.egg/EGG-INFO/requires.txt
google-api-python-client 1.7.8 is already the active version in easy-install.pth

Installed /Users/lovelife/anaconda3/lib/python3.6/site-packages/google_api_python_client-1.7.8-py3.6.egg
Processing dependencies for google-api-python-client
Finished processing dependencies for google-api-py

Requirement already up-to-date: google-cloud-language in /Users/lovelife/anaconda3/lib/python3.6/site-packages (1.1.1)


In [192]:

cred_file_loc = r'dogwood-dryad-235122-9721a66a61c1.json'
cred = service_account.Credentials.from_service_account_file(cred_file_loc)
Client = language.LanguageServiceClient(credentials=cred)
paragraphs = all_data_df['text'].tolist()
results = []
for paragraph in paragraphs:
    document = types.Document(content=paragraph, type=enums.Document.Type.PLAIN_TEXT)
    result = Client.analyze_sentiment(document=document)
    results.append(result)

results[11]

document_sentiment {
  magnitude: 0.30000001192092896
}
language: "en"
sentences {
  text {
    content: "Can I ask just sort of quick one?"
    begin_offset: -1
  }
  sentiment {
    magnitude: 0.20000000298023224
    score: 0.20000000298023224
  }
}
sentences {
  text {
    content: "Just for Greg."
    begin_offset: -1
  }
  sentiment {
  }
}
sentences {
  text {
    content: "Greg, when we think about the 727 engine versus the MAX, just for modeling purposes it\'s going to review 90% MAXs this year."
    begin_offset: -1
  }
  sentiment {
  }
}
sentences {
  text {
    content: "How do we think about the margin profile of one variant versus the other?"
    begin_offset: -1
  }
  sentiment {
  }
}

In [193]:
def classify_google(x):
    if x['score']<=-0.10:
        return 'negative'
    elif x['score']>-0.10 and (x['score']<0.25):
        return 'neutral'
    else:
        return 'positive'

magnitudes=[]
scores=[]
for i in range(len(results)):
    magnitudes.append(results[i].document_sentiment.magnitude)
    scores.append(results[i].document_sentiment.score)
google_sentiment=pd.DataFrame({'magnitude':magnitudes,'score':scores})
google_sentiment['label_goog']=google_sentiment.apply(classify_google,axis=1)
google_merged=pd.concat([all_data_df,google_sentiment],axis=1)
test_confusion_matrix=pd.DataFrame(confusion_matrix(google_merged['sentiment'].tolist(), google_merged['label_goog'].tolist()))
test_confusion_matrix.columns=confusion_columns=['predicted_neg','predicted_neutral','predicted_pos']
test_confusion_matrix['index']=['true_neg','true_neutral','true_pos']
test_confusion_matrix=test_confusion_matrix.set_index('index')
test_confusion_matrix.index.name=''

display(test_confusion_matrix)

,predicted_neg,predicted_neutral,predicted_pos
,,,
true_neg,23,105,27
true_neutral,56,639,140
true_pos,25,329,300


In [194]:
google_merged

,index,text,sentiment,magnitude,score,label_goog
0,1,Could you catch us up on your latest thinking ...,neutral,0.9,-0.1,negative
1,10,We continue to ramp up the activities for the ...,neutral,2.4,0.4,positive
...,...,...,...,...,...,...
1642,98,And so it's -- that one is a little bit toughe...,positive,0.8,0.4,positive
1643,99,"Thanks, Simeon. Rob, next question please?",neutral,0.1,0.0,neutral


In [195]:
google_merged.to_csv('google_API.csv')

In [78]:
(20+466+305)/1644

0.4811435523114355

# Amazon

In [196]:
for index, row in all_data_df.iterrows():
    text_file = open("Amazon/text_"+str(index)+".txt", "w")
    text_file.write(row['text'])
    text_file.close()

In [197]:
amazon_predict=pd.read_csv('amazon_prediction.csv')
amazon_predict['MyId']=amazon_predict['imagelocation'].apply(lambda x: int(x.split('_')[1].split('.')[0]))
amazon_predict=amazon_predict.sort_values(['MyId'])
amazon_predict=amazon_predict[['sentiment','positive','negative','neutral','mixed']]
amazon_predict['label_amazon_predicted']=amazon_predict['sentiment'].apply(lambda x: x.lower())
amazon_predict['label_amazon_predicted']=amazon_predict['label_amazon_predicted'].apply(lambda x: 'positive' if x=='mixed' else x)
amazon_predict=amazon_predict.drop('sentiment',axis=1)
amazon_predict

,positive,negative,neutral,mixed,label_amazon_predicted
985,0.008,2.155e-01,0.754,0.022,neutral
437,0.135,5.293e-04,0.862,0.002,neutral
...,...,...,...,...,...
1638,0.340,3.178e-02,0.083,0.545,positive
1425,0.058,3.285e-02,0.902,0.007,neutral


In [198]:
amzn_merged=pd.concat([all_data_df,amazon_predict],axis=1)
test_confusion_matrix=pd.DataFrame(confusion_matrix(amzn_merged['sentiment'].tolist(), amzn_merged['label_amazon_predicted'].tolist()))
test_confusion_matrix.columns=confusion_columns=['predicted_neg','predicted_neutral','predicted_pos']
test_confusion_matrix['index']=['true_neg','true_neutral','true_pos']
test_confusion_matrix=test_confusion_matrix.set_index('index')
test_confusion_matrix.index.name=''

display(test_confusion_matrix)

,predicted_neg,predicted_neutral,predicted_pos
,,,
true_neg,12,115,28
true_neutral,42,553,240
true_pos,41,524,89


In [199]:
google_merged_p4=google_merged.copy()
google_merged_p4['weighted_score_goog']=google_merged_p4['magnitude']*google_merged_p4['score']#[['']]
mag=google_merged_p4['weighted_score_goog'].max()-google_merged_p4['weighted_score_goog'].min()
google_merged_p4['weighted_score_goog']=google_merged_p4['weighted_score_goog']/(0.5*mag)
google_merged_p4

,index,text,sentiment,magnitude,score,label_goog,weighted_score_goog
0,1,Could you catch us up on your latest thinking ...,neutral,0.9,-0.1,negative,-0.029
1,10,We continue to ramp up the activities for the ...,neutral,2.4,0.4,positive,0.314
...,...,...,...,...,...,...,...
1642,98,And so it's -- that one is a little bit toughe...,positive,0.8,0.4,positive,0.105
1643,99,"Thanks, Simeon. Rob, next question please?",neutral,0.1,0.0,neutral,0.000


In [200]:
amazon_predict_p4=amazon_predict.copy()
#amazon_predict_p4.ix['combined_score','label_amazon_predicted']
amazon_predict_p4.loc[amazon_predict_p4['label_amazon_predicted']=='positive', 'weighted_score_amzn'] =amazon_predict_p4.loc[amazon_predict_p4['label_amazon_predicted']=='positive', 'positive']
amazon_predict_p4.loc[amazon_predict_p4['label_amazon_predicted']=='negative', 'weighted_score_amzn'] =-amazon_predict_p4.loc[amazon_predict_p4['label_amazon_predicted']=='negative', 'negative']
amazon_predict_p4.loc[amazon_predict_p4['label_amazon_predicted'].isin(['neutral','mixed']), 'weighted_score_amzn'] =0
mag=amazon_predict_p4['weighted_score_amzn'].max()-amazon_predict_p4['weighted_score_amzn'].min()
amazon_predict_p4['weighted_score_amzn']=amazon_predict_p4['weighted_score_amzn']/(mag)
amazon_predict_p4


,positive,negative,neutral,mixed,label_amazon_predicted,weighted_score_amzn
985,0.008,2.155e-01,0.754,0.022,neutral,0.00
437,0.135,5.293e-04,0.862,0.002,neutral,0.00
...,...,...,...,...,...,...
1638,0.340,3.178e-02,0.083,0.545,positive,0.17
1425,0.058,3.285e-02,0.902,0.007,neutral,0.00


In [201]:
amazon_predict_p4.to_csv('amazon_API.csv')

# Mircro

In [204]:
def classify_combined_score(x):
    if x>=0.1:
        return 'positive'
    elif x>-0.01 and x<0.10:
        return 'neutral'
    else:
        return 'negative'
all_merged=pd.concat([google_merged_p4[['text','sentiment','weighted_score_goog']],amazon_predict_p4[['weighted_score_amzn']]],axis=1)
all_merged['combined_score']=all_merged[['weighted_score_goog','weighted_score_amzn']].mean(axis=1)
all_merged['combined_sentiment']=all_merged['combined_score'].apply(classify_combined_score)
all_merged.groupby(['combined_sentiment'])[['text']].count()
test_confusion_matrix=pd.DataFrame(confusion_matrix(all_merged['sentiment'].tolist(), all_merged['combined_sentiment'].tolist()))
test_confusion_matrix.columns=confusion_columns=['predicted_neg','predicted_neutral','predicted_pos']
test_confusion_matrix['index']=['true_neg','true_neutral','true_pos']
test_confusion_matrix=test_confusion_matrix.set_index('index')
test_confusion_matrix.index.name=''

display(test_confusion_matrix)

,predicted_neg,predicted_neutral,predicted_pos
,,,
true_neg,23,98,34
true_neutral,51,517,267
true_pos,51,358,245


In [ ]:
! pip install auto-sklearn
! pip install -U scikit-learn
! pip install numpy
! pip install -U numpy
! pip install --upgrade pip